# Tanzania Water Wells Project

Morgan Nash

August 2025

# Overview

## Business Understanding

clean drinking water, look deeper to see looking at different features like pump creation or water to predict pump is functional or needs repair



In [1]:
### copied but helpful: """"With the ongoing global climate change, water scarcity has become an increasing problem in Tanzania, as some regions experience either intense and destructive rainfall or long dry spells.

###According to the World Bank, only 61% of households in Tanzania currently have access to a basic water-supply, 32% have access to basic sanitation, and 48% have access to basic hygiene. About 31,000 deaths each year emanate from lack of or poor quality water and sanitation, which is costly to the economy.

###The average water project in Tanzania about USD 6,000 to USD 8,000 for a hand pump water point, impacting 600-700 lives per well. For mechanized systems, the cost is atleast USD 12,000.

###The above cost is too high for a developing country, and given that up to 90% of pumps and other water extraction equipment in Tanzania fail due to lack of repair and maintenance, the country can only provide water to more millions of its people by following effective installation methods and proper maintenance of existing water projects.

###I intend to partner with the Government of Tanzania in solving the clean and safe water crisis."""""  

###Is it possible to correctly predict whether a pump is functional, functional-but-needs-repair, or non-functional given data such as the kind of pump, when it was installed, the installer, the region, and so on?
###Which are the 20 most important factors that influence whether a pump is functional, functional-but-needs-repair, or non-functional?
###What is the geographical distribution of pumps in terms of the three classes of functional, functional-but-needs-repair, and non-functional?
###Is there a relationship between a pump's total static head and its reliability?
###What is the relationship between installation year and a pump's functionality?
###What is the relationship between population and a pump's functionality?

## Data Understanding

The data for this project comes from


* id: Unique identifier for each water pump
* amount_tsh: Total static head (amount of water available to pump)
* date_recorded: Date the pump data was recorded
* funder: Person or org funded the pump installation
* gps_height: Altitude of the pump location
* installer: Person or org that installed the pump
* longitude: GPS longitude coordinate of the pump location
* latitude: GPS latitude coordinate of the pump location
* wpt_name: Name of the waterpoint (if available)
* num_private: Number of private plots reserved for the waterpoint
* basin: Geographic basin of the pump location
* subvillage: Geographic location within the village
* region: Geographic location
* region_code: Coded- geographic region
* district_code: Coded- administrative district
* lga: Geographic location (Local Government Area)
* ward: Geographic lovation (Administrative division)
* population: Population served by the well
* public_meeting: T/F Indicator of whether there was a public meeting about the well
* recorded_by: Group entering this row of data
* scheme_management: Who operates the waterpoint
* scheme_name: Who operates the waterpoint
* permit: Indicator of whether the waterpoint is permitted
* construction_year: Year the pump was installed
* extraction_type: The kind of extraction the waterpoint uses
* extraction_type_group: The kind of extraction the waterpoint uses
* extraction_type_class: The kind of extraction the waterpoint uses
* management: Type of management for the pump
* management_group: Grouped management type
* payment: Payment type for water service
* payment_type: Type of payment
* water_quality: Quality of water provided by the pump
* quality_group - The quality of the water
* quantity - The quantity of water
* quantity_group - The quantity of water
* source - The source of the water
* source_type - The source of the water
* source_class - The source of the water
* waterpoint_type - The kind of waterpoint
* waterpoint_type_group - The kind of waterpoint

# Exploratory Data Analysis

## Data Preparation & Cleaning

In [2]:
#import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

In [3]:
#load the data
val = pd.read_csv('data/Training_set_values.csv')
val.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
#Initially looks like there's many categorical columns that will need to be OneHotEncoded!

In [5]:
#load the data
labels = pd.read_csv('data/Training_set_labels.csv')
labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [6]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [7]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [8]:
labels['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [9]:
# Check class balance with percentages: Pretty significant imbalance!!
labels['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [10]:
# make problem binary: change the status_group labels 'non functional' and 'functinoal needs repair' into 'needs repair'
labels['status_group'] = labels['status_group'].map({'non functional':'needs repair',
                                                       'functional needs repair':'needs repair',
                                                       'functional':'functional'})

In [11]:
labels['status_group'].value_counts()

functional      32259
needs repair    27141
Name: status_group, dtype: int64

In [12]:
#although not all, this fixes a good amount of class imbalance
labels['status_group'].value_counts(normalize=True)

functional      0.543081
needs repair    0.456919
Name: status_group, dtype: float64

In [13]:
# Merge the two dataframes based the common index column 'id'
df = pd.merge(val, labels, how = 'left', on='id')
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,needs repair
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [14]:
#Note: many columns seem to contain the same/similar information

In [15]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [17]:
#Take a look at Summary Statistics for Numerical and Categorical Columns:
print("Numerical Feature Summary Statistics:")
numerical_stats = df.describe()
print(numerical_stats)
print("\n")
    

print("Summary statistics of categorical features:")
categorical_stats = df.describe(include=['object'])
print(categorical_stats)

Numerical Feature Summary Statistics:
                 id     amount_tsh    gps_height     longitude      latitude  \
count  59400.000000   59400.000000  59400.000000  59400.000000  5.940000e+04   
mean   37115.131768     317.650385    668.297239     34.077427 -5.706033e+00   
std    21453.128371    2997.574558    693.116350      6.567432  2.946019e+00   
min        0.000000       0.000000    -90.000000      0.000000 -1.164944e+01   
25%    18519.750000       0.000000      0.000000     33.090347 -8.540621e+00   
50%    37061.500000       0.000000    369.000000     34.908743 -5.021597e+00   
75%    55656.500000      20.000000   1319.250000     37.178387 -3.326156e+00   
max    74247.000000  350000.000000   2770.000000     40.345193 -2.000000e-08   

        num_private   region_code  district_code    population  \
count  59400.000000  59400.000000   59400.000000  59400.000000   
mean       0.474141     15.297003       5.629747    179.909983   
std       12.236230     17.587406       9.6

In [18]:
# Look at categorical columns
df.select_dtypes(include=['object']).columns

Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group'],
      dtype='object')

In [19]:
#compare columns that seem to have overlap and decide which to keep/which to get rid of (ensuring no multicolinearity)

In [20]:
df[['management_group', 'management']].value_counts()


management_group  management      
user-group        vwc                 40507
                  wug                  6515
                  water board          2933
                  wua                  2535
commercial        private operator     1971
parastatal        parastatal           1768
commercial        water authority       904
other             other                 844
commercial        company               685
unknown           unknown               561
other             other - school         99
commercial        trust                  78
dtype: int64

In [21]:
df[['waterpoint_type_group', 'waterpoint_type']].value_counts()

waterpoint_type_group  waterpoint_type            
communal standpipe     communal standpipe             28522
hand pump              hand pump                      17488
other                  other                           6380
communal standpipe     communal standpipe multiple     6103
improved spring        improved spring                  784
cattle trough          cattle trough                    116
dam                    dam                                7
dtype: int64

In [22]:
df[['funder', 'installer']].value_counts()

funder                          installer           
Government Of Tanzania          DWE                     4254
                                Government              1607
Hesawa                          DWE                     1296
Danida                          DANIDA                  1046
Rwssp                           DWE                      914
                                                        ... 
Ola                             OLS                        1
Okutu Village Community         Gerald Mila                1
Oikos E.Africa/ European Union  Oikos E .Africa            1
Oda                             VILLAGE COUNCIL .ODA       1
Msikiti Masji                   ms                         1
Length: 3697, dtype: int64

In [23]:
df[['scheme_management', 'scheme_name']].value_counts()

scheme_management  scheme_name               
VWC                K                             571
WUA                Chalinze wate                 404
VWC                DANIDA                        378
                   M                             331
WUG                None                          325
                                                ... 
VWC                NKENJA                          1
                   NYA/ MAK/ BUK piped scheme      1
                   NYA/MAK/BUK  piped scheme       1
                   NYA/MAK/BUK piped scheme        1
Company            BFFS                            1
Length: 3077, dtype: int64

In [24]:
df[['extraction_type_class', 'extraction_type_group' , 'extraction_type']].value_counts()

extraction_type_class  extraction_type_group  extraction_type          
gravity                gravity                gravity                      26780
handpump               nira/tanira            nira/tanira                   8154
other                  other                  other                         6430
submersible            submersible            submersible                   4764
handpump               swn 80                 swn 80                        3670
motorpump              mono                   mono                          2865
handpump               india mark ii          india mark ii                 2400
                       afridev                afridev                       1770
submersible            submersible            ksb                           1415
rope pump              rope pump              other - rope pump              451
handpump               other handpump         other - swn 81                 229
wind-powered           wind-powered  

In [25]:
df[['payment_type', 'payment']].value_counts()

payment_type  payment              
never pay     never pay                25348
per bucket    pay per bucket            8985
monthly       pay monthly               8300
unknown       unknown                   8157
on failure    pay when scheme fails     3914
annually      pay annually              3642
other         other                     1054
dtype: int64

In [26]:
df[['quality_group', 'water_quality']].value_counts()

quality_group  water_quality     
good           soft                  50818
salty          salty                  4856
unknown        unknown                1876
milky          milky                   804
colored        coloured                490
salty          salty abandoned         339
fluoride       fluoride                200
               fluoride abandoned       17
dtype: int64

In [27]:
df[['source_class', 'source_type', 'source']].value_counts()

source_class  source_type           source              
groundwater   spring                spring                  17021
              shallow well          shallow well            16824
              borehole              machine dbh             11075
surface       river/lake            river                    9612
              rainwater harvesting  rainwater harvesting     2295
groundwater   borehole              hand dtw                  874
surface       river/lake            lake                      765
              dam                   dam                       656
unknown       other                 other                     212
                                    unknown                    66
dtype: int64

* extraction_type_group: Grouped extraction type (redundant with 'extraction_type').
* extraction_type_class: Classification of extraction type (redundant with 'extraction_type').
* management_group: Grouped management type (redundant with 'management').
* payment_type: Payment method (redundant with 'payment').
* quality_group: Grouped water quality (redundant with 'water_quality').
* quantity_group: Grouped water quantity (redundant with 'quantity').
* source_type: Source type (redundant with 'source').
* source_class: Source class (redundant with 'source').
* waterpoint_type_group: Grouped waterpoint type (redundant with 'waterpoint_type').


In [28]:
categorical = [var for var in df.columns if df[var].dtype=='O']
categorical

['date_recorded',
 'funder',
 'installer',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'lga',
 'ward',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group',
 'status_group']

In [29]:
numerical = [var for var in df.columns if df[var].dtype!='O']
numerical

['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year']

## Modeling

# Conclusions

## Limitations

## Recommendations

## Next Steps